<div align="center">
<h1 align="center"><strong>Does size matter?</strong></h1>
  <p align="center">
    Applied Data Analysis (CS-401)
  </p>
</div>

Customers are increasingly relying on **product rating** websites to inform their purchasing decisions. It has been demonstrated that when customers rate a product, they often exhibit a **tendency to be influenced by the previous ratings** of other customers, a phenomenon known as the **_herding effect_**.

Despite this, an unresolved research question revolves around comprehending **how ratings might be impacted by the scale and the reputation of the vendor**. Utilizing data sourced from beer reviews websites, our objective is to investigate the **connection** between the **size and fame of vendors** (specifically, breweries) and **the perceived quality** of their products.

Through the quantification of brewery size and popularity using **predefined metrics** and the **extraction of sentiment** from textual reviews, our aim is to ascertain whether a correlation exists between vendor size and notoriety and perceived product quality. Additionally, we plan to **explore the behaviors** of diverse consumer bases, considering **temporal dimensions** (how these phenomena have evolved over the years and seasons within the same year) and **spatial dimensions** (how these relationships differ across states and countries).


## **BeerAdvocate**: Project proposal and initial analysis

In [ ]:
# Import  libraries
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import seaborn as sns
import difflib
import os
from datetime import datetime

In [ ]:
# Define the data folder paths for BeerAdvocate
DATA_FOLDER_BA = '../DATA/BeerAdvocate/'

# Define the file paths for the datasets
Beers_DATASET = DATA_FOLDER_BA+"beers.csv"
Users_DATASET = DATA_FOLDER_BA+"users.csv"
Reviews_DATASET = DATA_FOLDER_BA+"reviews_BA.csv"
Breweries_DATASET = DATA_FOLDER_BA+"breweries.csv"

# Dataset exploration & cleaning

The first step of the project consits in exploring the data available in the dataset as well as pre-processing them. This step consists mainly in handling the potential missing data and reformatting what needs to be reformated. The following subsections present this step for every datasets available.

### **Beers**

We starts with the dataset containing information about beers. We will probably not use this datatset since our project focuses more on the reviews and breweries but we decided to still perform some pre-processing on it in the case where we suddenly need it during the analysis of Milestone 3. The dataset includes the following columns:

- `beer_id`: Identifier for the beer.
- `beer_name`: Name of the beer.
- `brewery_id`: Identifier for the brewery producing the beer.
- `brewery_name`: Name of the brewery.
- `style`: Beer style.
- `nbr_ratings`: Number of ratings received.
- `nbr_reviews`: Number of reviews.
- `avg`: Average rating.
- `ba_score`: BeerAdvocate score.
- `bros_score`: Bros score.
- `abv`: Alcohol by volume.
- `avg_computed`: Computed average rating.
- `zscore`: Z-score.
- `nbr_matched_valid_ratings`: Number of matched valid ratings.
- `avg_matched_valid_ratings`: Average of matched valid ratings.


In [ ]:
# Read the datasets into Pandas DataFrames
beers = pd.read_csv(Beers_DATASET)
# Display 2 random chosen samples of the set
display(beers)

We can see that the dataset is composed of 280'823 beers. However, we can see that some data are missing and that some beers have 0 reviews. Furthermore, we should check if there is some duplicates. To address these issues, the following pre-processing steps are applied:

- **Filtering Beers with Less than 5 Reviews**: Deleting beers with fewer than 5 reviews, as they may not be characteristic.

- **Handling Missing Values**: Dropping rows with NaN values in the `nbr_ratings` column.

- **Removing Duplicates**: Dropping duplicate entries based on the `beer_name` and `beer_id` columns.

- **Column Selection**: Dropping columns that won't be used in our analysis.

In [ ]:
# Set a minimum threshold for the number of reviews
MIN_NUMBER_OF_REVIEWS = 5

# Create a filtered copy of the 'beers' DataFrame with a minimum number of reviews
beers_filt = beers.copy(deep=True)
beers_filt = beers_filt[beers_filt['nbr_reviews'] >= MIN_NUMBER_OF_REVIEWS]

# Remove rows with missing values in the 'nbr_reviews' column
beers_filt = beers_filt[beers_filt['nbr_reviews'].notna()]

# Drop duplicate entries based on the 'beer_name' column
beers_filt = beers_filt.drop_duplicates(subset=['beer_name'])

# Calculate the number of duplicate entries based on 'beer_name' and 'beer_id'
dupli_name = np.sum(beers_filt.duplicated(subset=['beer_name']))
dupli_ID = np.sum(beers_filt.duplicated(subset=['beer_id']))

# Drop specific columns from the filtered DataFrame
beers_filt = beers_filt.drop(['zscore', 'nbr_matched_valid_ratings', 'avg_matched_valid_ratings', 'bros_score', 'ba_score'], axis=1)

# Display the filtered DataFrame
display(beers_filt)

# Print the number of duplicate entries for 'beer_name' and 'beer_id'
print(f'Number of duplicate beer name = {dupli_name}')
print(f'Number of duplicate beer ID = {dupli_ID}')


After pre-processing, the number of beers drops to 42'923.

### **Users**

We then move to the pre-processing of the dataset containing information about users. The dataset includes the following columns:

- `nbr_ratings`: Number of ratings made.
- `nbr_reviews`: Number of reviews done.
- `user_id`: Unique user identifier.
- `user_name`: User Name.
- `joined`:  Date of the sign up.
- `location`: Location of the User.


In [ ]:
# Read the datasets into Pandas DataFrames
users = pd.read_csv(Users_DATASET)
display(users)

The dataset is composed of 153'704 users. We can again see that some data are missing and that some users did not gave any review. To address these issues our pre-processing involves the following steps:

- **Filtering Users with 0 number of reviews**: Deleting users with 0 reviews, as they are not characteristic.

- **Handling Missing Values**: Dropping rows with NaN values in the `nbr_reviews`, `user_id`, `user_name` and `location` columns.

- **Check for Duplicated Users**: Check if there are multiple user with the same id.

- **Formatting the date**: Reformat the date in the column `joined` in UTC format.

In [ ]:
# Create a deep copy of the 'users' dataframe
users_filt = users.copy(deep=True)

# Check for duplicates based on user name and user ID
dupli_name = np.sum(users_filt.duplicated(subset=['user_name']))
dupli_ID = np.sum(users_filt.duplicated(subset=['user_id']))

# Remove users with 0 reviews and NaN as the number of reviews
users_filt = users_filt[users_filt['nbr_reviews'] >= 1]
users_filt = users_filt[users_filt['nbr_reviews'].notna()]

# Remove rows with NaN in 'user_id', 'user_name', and 'location'
users_filt = users_filt[users_filt['user_id'].notna()]
users_filt = users_filt[users_filt['user_name'].notna()]
users_filt = users_filt[users_filt['location'].notna()]

# Convert 'joined' column to datetime type
users_filt['joined'] = users_filt['joined'].apply(lambda x: datetime.utcfromtimestamp(x) if not pd.isna(x) else x)

# Rename the 'location' column to 'user_location'
users_filt.rename(columns={'location': 'user_location'}, inplace=True)

# Display the resulting dataframe
display(users_filt)

# Display the number of duplicate user names and user IDs
print(f'Number of duplicate user names = {dupli_name}')
print(f'Number of duplicate user IDs = {dupli_ID}')

# Display the number of NaN values in each category
print('Number of NaN by category:')
print(np.sum(users_filt.isna()))


After pre-processing, a total of 58'199 users with complete information are kept.

### **Breweries**

Next, we conduct the pre-processing of the dataset containing information about breweries. The dataset comprises the following columns:

- `id`: Brewery identifier.
- `location`: Location of the brewery.
- `name`: Name of the brewery.
- `nbr_beers`: Number of beers produced.


In [ ]:
# Read the datasets into Pandas DataFrames
breweries = pd.read_csv(Breweries_DATASET)
display(breweries)

The dataset is composed of 16'758 breweries. We can directly see that some breweries have 0 beers and are therefore not relevant for our analysis. There might be some missing values as well, thus we decided to perform the following pre_processing steps:

- **Filtering Breweries with 0 number of beers**: Deleting breweries with 0 beers, as they are not characteristic.

- **Handling Missing Values**: Dropping rows with NaN values in the `nbr_beers`.

- **Removing Duplicates**: Dropping duplicate entries based on the `name` column.

In [ ]:
# Create a deep copy of the 'breweries' dataframe
breweries_filt = breweries.copy(deep=True)

# Check for duplicates based on ID
dupli_ID = np.sum(breweries_filt.duplicated(subset=['id']))

# Remove breweries with 0 beers and NaN values
breweries_filt = breweries_filt[breweries_filt['nbr_beers'] >= 1]
breweries_filt = breweries_filt[breweries_filt.notna()]

# Remove duplicate entries based on brewery name
breweries_filt = breweries_filt.drop_duplicates(subset='name')

# Check for duplicates based on name
dupli_name = np.sum(breweries_filt.duplicated(subset=['name']))

# Rename columns for consistency
breweries_filt.rename(columns={'name': 'brewery_name', 'id': 'brewery_id', 'location': 'brewery_location'}, inplace=True)

# Display the resulting dataframe
display(breweries_filt)

# Display the number of duplicate names and IDs
print(f'Number of duplicate names = {dupli_name}')
print(f'Number of duplicate IDs = {dupli_ID}')

# Display the number of NaN values in each category
print('Number of NaN by category:')
print(np.sum(breweries_filt.isna()))


After pre-processing, 14'158 breweries are remaining. 

### **Reviews**

We now delve into the BeerAdvocate Reviews dataset, focusing on reviews of various beers. The dataset contains the following columns:

- `beer_name`: Name of the beer.
- `beer_id`: Identifier for the beer.
- `brewery_name`: Name of the brewery producing the beer.
- `brewery_id`: Identifier for the brewery.
- `style`: Beer style.
- `abv`: Alcohol by volume.
- `date`: Timestamp of the review.
- `user_name`: Username of the reviewer.
- `user_id`: Identifier for the user.
- `appearance`: Rating for the beer's appearance.
- `aroma`: Rating for the beer's aroma.
- `palate`: Rating for the beer's palate.
- `taste`: Rating for the beer's taste.
- `overall`: Overall rating.
- `rating`: Overall user rating.
- `text`: Review text.

Since the .txt file containing the reviews is huge, we first converted it into smaller files that we then regrouped in a .csv file. The code in `split_reviews.py` was used or this purpose.

In [ ]:
# Read the Reviews dataset into a pandas DataFrame.
reviews_BA = pd.read_csv(Reviews_DATASET)
display(reviews_BA)

Our pre-processing begins with the following initial steps:

- **Converting Timestamps to Datetime**: We start by converting the 'date' column, which contains timestamps, into the datetime format. This conversion enables us to perform time-based analyses more effectively.

- **Handling Missing Values**: We address missing values in the dataset by dropping rows with NaN values. This ensures that our analysis is based on complete and reliable data.

- **Removing Unnecessary Column**: The 'abv' column, representing the alcohol by volume, is not useful for our specific analysis. Consequently, we opt to drop this column to streamline our dataset.

- **Removing White Space before and after Stings**: Some strings columns like `user_name` begin with a white space, which is a problem for merging. Consequently we need to remove them.

In [ ]:
# Create a deep copy of the reviews_BA DataFrame to avoid modifying the original DataFrame.
reviews_filt = reviews_BA.copy(deep=True)
# Convert the 'date' column to a datetime format.
# If the 'date' value is not NaN, apply the conversion using utcfromtimestamp.
# If the 'date' value is NaN, leave it unchanged.
reviews_filt['date'] = reviews_filt['date'].apply(lambda x: datetime.utcfromtimestamp(x) if not pd.isna(x) else x)
# Drop rows where the 'text' column has NaN values.
reviews_filt = reviews_filt[reviews_filt['text'].notna()]
# Drop the 'abv' column from the reviews_filt DataFrame.
reviews_filt = reviews_filt.drop(['abv'], axis=1)
# Remove leading and trailing whitespaces (if they exist) from the following columns: 
# user_id, user_name, beer_name, brewery_name, style, and text.
reviews_filt.user_id = reviews_filt.user_id.apply(lambda x: x.strip())
reviews_filt.user_name = reviews_filt.user_name.astype(str).apply(lambda x: x.strip())
reviews_filt.beer_name = reviews_filt.beer_name.apply(lambda x: x.strip())
reviews_filt.brewery_name = reviews_filt.brewery_name.apply(lambda x: x.strip())
reviews_filt['style'] = reviews_filt['style'].apply(lambda x: x.strip())
reviews_filt.text = reviews_filt.text.apply(lambda x: x.strip())
# Display the updated reviews_filt DataFrame.
display(reviews_filt)
# Print the number of NaN values for each column in the reviews_filt DataFrame.
print('Number of NaN by category:')
print(np.sum(reviews_filt.isna()))


We now aim to visualize the distribution of the length of the reviews to:

- **Get insight into review length variation**: Visualizing the distribution allows us to understand the range and variability in review lengths. Some reviews may be succinct, while others may be more detailed.

- **Assess data quality**: Analyzing review lengths can also serve as a quality check. Unusually short or long reviews may warrant further investigation to ensure data integrity.


In [ ]:
# Create a histogram of the review lengths using the 'text' column from the reviews_filt DataFrame.
plt.hist(reviews_filt['text'].str.len(), bins=200, log=True)

# Set x-axis and y-axis labels and the title.
plt.xlabel('Review length')
plt.ylabel('Number of reviews')
plt.title('Distribution of Review Length')

plt.show()


The BeerAdvocate website advises to create reviews of at list 150 characters. It can be seen that not all the reviews have at least 150 characters. We will therefore remove them in the next steps to keep only relevant revies. The statistics of the review length are displayed in the next cell. As we can see, the median is at 580 characters. The distribution is skewed though, with a small quantity of reviews being more than 5000 characters. This will be interesting to analyse in the next milestone if there is a link between the length of the reviews and the scale or popularity of a brewery.

In [ ]:
# Display descriptive statistics of the review lengths.
reviews_filt['text'].str.len().describe()

We attempted to improve consistency by **translating** all non-English textual reviews. To this end, we used the language detection module, $\texttt{detect}$, of the $\texttt{langdetect}$ library to **initially identify the language of each review**.

Due to the considerable computation time required for language detection, we decided to **keep the language identifier** of each review in a separate dataset, together with the **unique identifiers** of the **beer** and the **user**.

This approach allows us to store the language information in our archive, facilitating efficient access without the need to calculate the language detection for each review each time.

Note that if the CSV file containing `user_id`, `beer_id` and `text_lang` (the language identifier of the reviews) exists in the repository, we can avoid recomputing the information. Instead, we can merge the review dataset with this auxiliary dataset into a consolidated dataset, simplifying our analysis process.

In [ ]:
# The next sections will need these modules to be run
# Importing the 'unescape' function from the 'html' module for text cleaning of html escape characters
try:
    from html import unescape
except:
    !pip install html

# Importing the 'detect' function from the 'langdetect' module for language detection of reviews
try:
    from langdetect import detect
except:
    !pip install langdetect

# Importing the 'GoogleTranslator' from the 'deep_translatore' module for reviews translation
try:
    from deep_translator import GoogleTranslator
except:
    !pip install deep-translator

In [ ]:
# If the .csv file exists then we don't redo the detection
data_name = 'reviews_lang.csv'
CODE_ERROR = 'Error'

if os.path.exists(data_name):
    reviews_language = pd.read_csv(data_name)
    reviews_filt = pd.merge(reviews_filt, reviews_language, on=['beer_id','user_id'], how='left')
else:
    # Detect the language of each review. Handle exception for non corrected reviews.
    text_lang = []
    for review in reviews_filt['text']:
        try:
            text_lang.append(detect(review))
        except:
            text_lang.append(CODE_ERROR)
            continue
    
    # Adding a new column 'text_lang' to store the detected language for each review
    reviews_filt['text_lang'] = pd.Series(text_lang)

    # Store the language information in a  
    reviews_filt[['beer_id', 'user_id', 'text_lang']].to_csv(data_name, index=False)

We can look at the consolidated dataset with the new column giving the language of the review.

In [ ]:
# display 3 randomly chosen rows of the new consolidated dataset
display(reviews_filt.sample(3))

Now we can check if errors occured during the detection, as well as if NaN values appeared in the process.

In [ ]:
print('The number of errors detected are:', np.sum(reviews_filt['text_lang'] == CODE_ERROR))

In [ ]:
print(f'The number of NaN values in text_lang colum is:', np.sum(reviews_filt['text_lang'].isna()), '/', len(reviews_filt['text_lang']))

We then decide to drop the NaN values due to the small number of occurrences of NaN values in the `text_lang` column in our filtered reviews dataset.

In [ ]:
reviews_filt = reviews_filt[reviews_filt['text_lang'].notna()]

We can now have a look to the variety of languages composing the reviews. As expected a vast majority of them are in English.

In [ ]:
print('The count and variety of distinct languages used in the reviews within our dataset:')
print(reviews_filt['text_lang'].value_counts())

We then filter rows in the DataFrame where the `text_lang` column is not 'en', and then apply translation to English for the corresponding 'text' column.

In [ ]:
"""
    Translates the input text to English.

    Parameters:
        text (str): The input text to be translated.
        source (str): The source language (default is 'auto' for automatic detection).
        target (str): The target language (default is 'en' for English).

    Returns:
        The translated text in English.
"""
def translate_to_english(text, source='auto', target='en'):
    translated = GoogleTranslator(source=source, target=target).translate(text)
    return translated

In [ ]:
reviews_filt[reviews_filt.text_lang != 'en']['text'] = reviews_filt[reviews_filt.text_lang != 'en']['text'].apply(lambda x: translate_to_english(x))

With all reviews now in **English**, we can proceed to remove the `text_lang` column.

In [ ]:
reviews_filt = reviews_filt.drop(['text_lang'], axis=1)

We then undertook further pre-processing, focusing on the textual representation of reviews. We followed the following steps:

- **Management of special characters**: After examining the modified and translated dataset, we observed the presence of some special characters such as "\&quot;" and "\x92" in some reviews. To solve this problem, we used the html.unescape function to convert the HTML entities and then removed the non-ASCII characters by encoding them in ASCII and decoding them again.

- **Filtering short reviews**: As a final step, we filtered out reviews with less than 150 characters. This step aimed to exclude shorter reviews from our dataset, focusing on more substantial texts.

In [ ]:
# Apply the 'unescape' function to decode HTML entities in the 'text' column
reviews_filt['text'] = reviews_filt['text'].apply(unescape)

# Remove non-ASCII characters by encoding to ASCII and decoding back
reviews_filt['text'] = reviews_filt['text'].apply(lambda x: x.encode('ascii', 'ignore').decode())

# Set the minimum number of characters for reviews
MIN_NUMBER_OF_CHARACTER = 150

# Filter out reviews with fewer than 'min_character' characters
reviews_filt = reviews_filt[reviews_filt['text'].str.len() > MIN_NUMBER_OF_CHARACTER]

In [ ]:
display(reviews_filt)

We end up with 2'587'598 reviews. Now that the datasets are ready, we can start with some preliminary analysis.

## **Metrics Definition**
In our analysis, we introduce **two key metrics** to quantitatively **assess the characteristics of breweries**: Size Metrics and Popularity Metrics. These metrics are created in the next cells. Since we don't have data about the revenue or number of liter produced by the brewery, we decided to base ourself on variable present in the dataset to construct these metrics. Some verification will be done by hand, by searching information about some breweries on the web to assess the quality of the metrics and to correct the coefficients if needed.

####  **Size Metrics**
To numerically evaluate the size of a brewery, we built an index based on the following formula:

$$ \text{Size} = \alpha N_r + \beta N_b + \gamma N_t$$

With:
- $N_r =$ number of reviews normalized by the total number of reviews
- $N_b =$ number of beers produced normalized by the total number of beers
- $N_t =$ number of different types (style) of beer produced normalized by the total number of styles

The coefficients $\alpha, \beta, \gamma$ are chosen by hand. 

#### **Popularity Metrics**
$$ \text{Popularity} = \dfrac{N_r}{N_b}$$

With $N_r =$ number of reviews, $N_b =$ number of beers produced


Note that both metrics are normalized to get a value between 0 and 1.

In [ ]:
# Extract brewery_id and style columns from reviews_filt and merge with breweries_filt to get the type of the beer reviewed
review_brew = reviews_filt[['brewery_id', 'style']].copy(deep=True)
review_brew = pd.merge(review_brew, breweries_filt, on=['brewery_id'], how='inner')

# Drop brewery location column from the merged DataFrame as we're not planning to use it
review_brew = review_brew.drop(['brewery_location'], axis=1)

# Count the number of reviews for each brewery
brew_review_count = review_brew.groupby('brewery_id').size().reset_index(name='nbr_reviews')

# Count the number of unique styles for each brewery
brew_style_count = review_brew.groupby('brewery_id')['style'].unique().reset_index(name='styles')
brew_style_count['nbr_styles'] = brew_style_count['styles'].apply(lambda x: len(x))

# Merge the review count, style count, and selected columns from breweries_filt
df_metrics = pd.merge(brew_review_count, brew_style_count[['brewery_id', 'nbr_styles']], on=['brewery_id'], how='inner')
df_metrics = pd.merge(df_metrics, breweries_filt[['brewery_id', 'brewery_name', 'nbr_beers']], on=['brewery_id'], how='inner')

# Compute size metrics
tot_nb_reviews = df_metrics['nbr_reviews'].sum()
tot_nb_beers = df_metrics['nbr_beers'].sum()
tot_nb_styles = df_metrics['nbr_styles'].sum()
# Choose coeff. aplha, beta, gamma for size metrics
alpha = 3
beta = 1
gamma = 1

# Calculate size metrics based on the weighted sum of reviews, beers, and styles
df_metrics['size_metrics'] = df_metrics.apply(lambda row: alpha * row['nbr_reviews'] / tot_nb_reviews + beta * row['nbr_beers'] / tot_nb_beers + gamma * row['nbr_styles'] / tot_nb_styles, axis=1)

# Normalize size metrics to ensure values are between 0 and 1
df_metrics['size_metrics'] = df_metrics['size_metrics'] / df_metrics['size_metrics'].max()

# Calculate popularity metrics based on the ratio of reviews to beers
df_metrics['popularity_metrics'] = df_metrics.apply(lambda row: (row['nbr_reviews'] / tot_nb_reviews) / (row['nbr_beers'] / tot_nb_beers), axis=1)

# Normalize popularity metrics to ensure values are between 0 and 1
df_metrics['popularity_metrics'] = df_metrics['popularity_metrics'] / df_metrics['popularity_metrics'].max()


Before looking at the metrics, we can have a look to the distribution of the variable used to compute them.

In [ ]:
# Create subplots
fig, axs = plt.subplots(1, 3, figsize=(12, 4))

# Plot histogram for '# reviews' in the first subplot
axs[0].hist(df_metrics['nbr_reviews'], bins=300, color='blue', alpha=0.7)
axs[0].set_title('Distribution of the breweries \n based on number of reviews')
axs[0].set_xlabel('Number of Reviews')
axs[0].set_ylabel('Number of breweries')

# Plot histogram for '# beers' in the second subplot
axs[1].hist(df_metrics['nbr_beers'], bins=300, color='green', alpha=0.7)
axs[1].set_title('Distribution of the breweries \n based on number of different beers')
axs[1].set_xlabel('Number of Different Beers')
axs[1].set_ylabel('Number of breweries')

# Plot histogram for '# style' in the third subplot
axs[2].hist(df_metrics['nbr_styles'], bins=50, color='red', alpha=0.7)
axs[2].set_title('Distribution of the breweries \n based on number of beer styles')
axs[2].set_xlabel('Number of Different Styles')
axs[2].set_ylabel('Number of breweries')

# Set y-axis scale to logarithmic for better visibility of distribution in all subplots
axs[0].set_yscale('log')
axs[1].set_yscale('log')
axs[2].set_yscale('log')

# Adjust layout
plt.tight_layout()

plt.show()

We can see that all the variables have similar distribution. The number of reviews seems to be more skewed though, with a few breweries having a lot more reviews. Note that the y-axis scale is set to **logarithmic** for better visualization of the distribution, and the layout is adjusted for better presentation. We can now have a look to how the breweries are distributed based on the metrics calculated.

In [ ]:
# Create subplots
fig, axs = plt.subplots(1, 2, figsize=(12, 4))

# Plot histogram for 'size metrics' in the first subplot
axs[0].hist(df_metrics['size_metrics'], bins=300, color='blue', alpha=0.7)
axs[0].set_title('Distribution of the breweries based on size metrics')
axs[0].set_xlabel('Size metrics')
axs[0].set_ylabel('Number of breweries')

# Plot histogram for 'popularity metrics' in the second subplot
axs[1].hist(df_metrics['popularity_metrics'], bins=300, color='green', alpha=0.7)
axs[1].set_title('Distribution of the breweries based on popularity metrics')
axs[1].set_xlabel('Popularity metrics')
axs[1].set_ylabel('Number of breweries')

# Set y-axis scale to logarithmic for better visibility of distribution in both subplots
axs[0].set_yscale('log')
axs[1].set_yscale('log')

# Adjust layout
plt.tight_layout()

plt.show()


We retrieve the same skew distribution for both metrics. But let's see if the biggest breweries are the also the most popular ones.

In [ ]:
# Sort breweries by size
size_sorted = df_metrics.sort_values(by='size_metrics', ascending=False).reset_index()
size_sorted.index = np.linspace(1, len(size_sorted), len(size_sorted)).astype(int)
print("Overview of the size metrics ranking:")

# Display a summary of the top breweries based on size metrics
display(size_sorted[['brewery_name', 'size_metrics', 'nbr_reviews', 'nbr_beers', 'nbr_styles']].iloc[[0, 1, 2, 9, 99, 999, 9999]])


# Sort breweries by popularity
popu_sorted = df_metrics.sort_values(by='popularity_metrics', ascending=False).reset_index()
popu_sorted.index = np.linspace(1, len(popu_sorted), len(popu_sorted)).astype(int)
print("Overview of the popularity metrics ranking:")

# Display a summary of the top breweries based on popularity metrics
display(popu_sorted[['brewery_name', 'popularity_metrics', 'nbr_reviews', 'nbr_beers']].iloc[[0, 1, 2, 9, 99, 999, 9999]])


When sorting the breweries by size, it becomes evident that the top-ranking breweries are primarily located in the US. This observation aligns with expectations, considering that the majority of reviewers and breweries are located in the US (as detailed in the following section).

Looking for the 3 biggest and the 100th on the web, we can extract their beer production:

- (1st) Boston Beer Company: 5,300,000 barrels (6,200,000 hL) found under https://en.wikipedia.org/wiki/Boston_Beer_Company
- (2nd) Sierra Nevada Brewing Co.: 1,250,000 barrels (510,000 hL) found under https://en.wikipedia.org/wiki/Sierra_Nevada_Brewing_Company
- (3rd) Stone Brewing: 325,645 barrels (382,000 hL) found under https://en.wikipedia.org/wiki/Stone_Brewing_Co.
- (100th) Uinta Brewing Company: 77'000 barrels found under https://en.wikipedia.org/wiki/Uinta_Brewing_Company

Given these values, it seems that the size metrics works, at least for the top ranked breweries.

On the other side, the popularity metrics yield significantly different results. The top-ranked breweries include historically renowned Belgian breweries such as "Brasserie de Rochefort," notable for its "Trappist" beer, and "Brasserie d'Orval." This highlights that the popularity metric captures a different aspect of brewery influence, focusing on factors beyond sheer size.

It is important to note that this preliminary analysis does not take into account the time evolution. It will be interesting to see how it has evolved over the years. Some breweries for example could have significantly changed size over the years. However this pre-analysis validates the choice of the metrics and opens a large range of possibilities for the next analysis.


## **Geographical Analysis**

The goal of this section is to gain insights into the geographical distribution of both breweries and reviewers within the dataset. Ultimately, we aim to calculate the distances between breweries and their respective reviewers. This analysis could potentially unveil distinctions between brewery types, revealing whether certain types of breweries attract predominantly local reviewers or have a more globally dispersed audience

### **Analysis of Brewery Location**

We initiate our analysis by examining the geographical distribution of breweries in the dataset. To achieve this, we integrate the dataset with a map sourced from Geopandas (https://www.naturalearthdata.com/downloads/110m-cultural-vectors/).

Ensuring alignment between the country names used in the map and those in the brewery dataset is crucial. To address this, we calculate the Hamming distance between them and substitute the brewery location with the closest match. In instances where no match is found, we opt to eliminate the corresponding brewery. This scenario applies to 26 breweries out of over 14,000, which is deemed acceptable for this level of analysis. It's worth noting that some removed breweries had HTTP links as their location, explaining the lack of match with the map.

For breweries located in the USA, the dataset includes information about the state. Consequently, we extend the same process to the states in the United States of America. To facilitate this, we introduce a new column, 'state,' in the dataframe. This allows us to split the location of US breweries into 'country' and 'state.' 

The geodataframe for the USA can be accessed here: https://eric.clst.org/tech/usgeojson/

In [ ]:
# Hamming distance function
def find_closest_match(input_str, target_list):
    """Find the closest match in a list using Hamming distance."""
    return difflib.get_close_matches(input_str, target_list, n=1, cutoff=0.5)

In [ ]:
# Make deep copy of breweries filtered dataframe reset indexes (needed for the loop next)
breweries_loc = breweries_filt.copy(deep=True)
breweries_loc.reset_index(inplace=True,drop=True)

# Load the world map shapefile
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Load the USA state map
us_states = gpd.read_file('GeoJSON/gz_2010_us_040_00_500k.json')

# Extract countries name
unique_loc = world['name'].unique()
# Extract state name
unique_state = us_states['NAME'].unique()

# Create column for state in breweries df, will be used for USA
breweries_loc['brewery_state'] = '-'

# Init dropped breweries
dropped_brew = []

# Loop through the 'brewery_location' column and replace values if needed
for i, location in enumerate(breweries_loc['brewery_location']):
    # Check if in USA --> (USA, state)
    try:
        location, state = location.split(', ')
    except:
        state = location  # if not in USA, state = location
    
    # Find closest match with Hamming distance
    closest_match_country = find_closest_match(location, unique_loc)
    
    if closest_match_country:
        # If closest match is USA --> match state      
        if closest_match_country[0] == 'United States of America':
            breweries_loc.at[i, 'brewery_location'] = closest_match_country[0]
            
            # Find state
            closest_match_state = find_closest_match(state, unique_state)
            if closest_match_state:
                breweries_loc.at[i, 'brewery_state'] = closest_match_state[0]
            else:
                breweries_loc = breweries_loc.drop(index=i)
        else:
            # If not in USA --> state = location 
            breweries_loc.at[i, 'brewery_location'] = closest_match_country[0]
            breweries_loc.at[i, 'brewery_state'] = closest_match_country[0]
    else:
        #print(i)
        #print(breweries_loc.at[i, 'brewery_location'])
        # If no match, drop
        dropped_brew.append(location)
        breweries_loc = breweries_loc.drop(index=i)

# Display a sample of the modified dataframe
display(breweries_loc.sample(3))

# Print the number of dropped breweries and their locations
print(f'{len(dropped_brew)} breweries were dropped, at the following location:')
print(dropped_brew)


We explore the **Top 10 countries** with the **highest number of breweries** to gain insights into the **global distribution of brewing establishments**.

In [ ]:
# Count the number of breweries in each country
brewery_counts = breweries_loc['brewery_location'].value_counts().reset_index()
brewery_counts.columns = ['brewery_location', 'nb_breweries']

# Identify the top 10 countries
top10 = brewery_counts.sort_values(by='nb_breweries', ascending=False).head(10)
top10 = top10[['brewery_location', 'nb_breweries']]
top10.index = np.linspace(1, 10, 10).astype(int)
top10.nb_breweries = top10.nb_breweries.astype(int)

# Display the Top 10 countries with the most breweries
print('Top 10 countries with the most breweries:')
display(top10)

As evident from the data, **there are significantly more breweries in the US** compared to other countries. Due to this notable concentration, we will consider the breweries by states in the next section. To facilitate this, we extend the world geodataframe by incorporating the US state geodataframe in the next section.

In [ ]:
# Add US states to world df
us_states.rename(columns={'NAME': 'name'}, inplace = True)
world_with_US_states = pd.concat([world[['name','geometry']],us_states[['name','geometry']]])

Let's look again to the Top 10 countries with the highest number of breweries. However, this time, we'll consider **each US state as a distinct 'country'.** Additionally, we'll **visualize the global distribution of breweries** to gain a comprehensive understanding of their geographical spread.

In [ ]:
# Count the number of breweries in each country
brewery_counts = breweries_loc['brewery_state'].value_counts().reset_index()
brewery_counts.columns = ['brewery_state', 'nb_breweries']

# Merge brewery counts with the world map data
world_merge = world_with_US_states.merge(brewery_counts, how='left', left_on='name', right_on='brewery_state')

# Fill NaN values (countries without breweries) with 0
world_merge['nb_breweries'].fillna(0, inplace=True)
# Fill NaN values of state with Unknown
world_merge['brewery_state'].fillna('Unknown', inplace=True)

# Find top 10 countries
top10 = world_merge.sort_values(by='nb_breweries', ascending=False).head(10)
top10 = top10[['name', 'nb_breweries']]
top10.index = np.linspace(1, 10, 10).astype(int)
top10.nb_breweries = top10.nb_breweries.astype(int)

# Display the Top 10 countries (considering US states) with the most breweries
print('Top 10 countries (considering US states) with the most breweries:')
display(top10)

# Plot the choropleth map
fig, ax = plt.subplots(1, 1, figsize=(15, 7))
world_merge.boundary.plot(ax=ax, color='black', linewidth=0.5)
world_merge.plot(column='nb_breweries', ax=ax, legend=True,
                 norm=LogNorm(vmin=1, vmax=world_merge['nb_breweries'].max()),
                 legend_kwds={'label': "Number of Breweries by Country",
                              'orientation': "vertical"},
                 cmap='Reds')

# Remove the axis
ax.set_title('Distribution of the breweries in the World')
ax.set_axis_off()
plt.show()


The revised analysis reveals distinct results. Germany takes the lead, followed by Finland, and several US states are now present in the Top 10. The conclusion of this analysis is that there are many more breweries in the US than anywhere else in the world. Acknowledging this, it becomes essential to consider the US's internal diversity in subsequent analyses. Given the availability of information about individual US states, incorporating them into the analysis could provide valuable insights, offering a more nuanced understanding of the brewing landscape within the country.

### **Analysis of User Location**
Now that we have gained insights into the distribution of breweries worldwide, we can apply a **similar analysis** to the **user data**.

In [ ]:
# Create a copy of the filtered users dataset and reset indexes (needed for the loop next)
users_loc = users_filt.copy(deep=True)
users_loc.reset_index(inplace=True,drop=True)
# Create a column for the state in the users dataframe, to be used for USA
users_loc['user_state'] = '-'

# Initialize a list for dropped user locations
dropped_user = []

# Loop through the 'user_location' column and replace values if needed
for i, location in enumerate(users_loc['user_location']):
    try:
        location, state = location.split(', ')
    except:
        state = location
    
    # Find the closest match using Hamming distance
    closest_match_country = find_closest_match(location, unique_loc)

    # If there is a match
    if closest_match_country:
        if closest_match_country[0] == 'United States of America':
            users_loc.at[i, 'user_location'] = closest_match_country[0]
            
            # Find the state
            closest_match_state = find_closest_match(state, unique_state)
            if closest_match_state:
                users_loc.at[i, 'user_state'] = closest_match_state[0]
            else:
                users_loc = users_loc.drop(index=i)
        else:
            # Replace the value in the DataFrame
            users_loc.at[i, 'user_location'] = closest_match_country[0]
            users_loc.at[i, 'user_state'] = closest_match_country[0]
    # No match, drop the row
    else:
        dropped_user.append(location)
        users_loc = users_loc.drop(index=i)

# Display a sample of the modified users dataframe
display(users_loc.sample(3))

# Print the number of dropped users and their locations
print(f'{len(dropped_user)} users were dropped, at the following location:')
print(dropped_user)

Let's examine the **Top 10 countries** with the **highest number of reviewers** to gain insights into the global distribution of reviewers.

In [ ]:
# Count the number of reviewers in each country
reviewers_counts = users_loc['user_location'].value_counts().reset_index()
reviewers_counts.columns = ['user_location', 'nb_reviewers']

# Identify the top 10 countries according to the number of reviewers
top10 = reviewers_counts.sort_values(by='nb_reviewers', ascending=False).head(10)
top10 = top10[['user_location', 'nb_reviewers']]
top10.index = np.linspace(1, 10, 10).astype(int)
top10.nb_reviewers = top10.nb_reviewers.astype(int)

# Display the Top 10 countries with the most reviewers
print('Top 10 countries with the most reviewers:')
display(top10)

As observed, there is once again a **substantial number of reviewers in the US**. In the next section, we will **replicate the analysis**, this time considering reviewers based on **US states**.

In [ ]:
# Count the number of reviewers in each country
reviewers_counts = users_loc['user_state'].value_counts().reset_index()
reviewers_counts.columns = ['user_state', 'nb_reviewers']

# Merge reviewer counts with the world map data
world_merge = world_with_US_states.merge(reviewers_counts, how='left', left_on='name', right_on='user_state')

# Fill NaN values (countries without reviewers) with 0
world_merge['nb_reviewers'].fillna(0, inplace=True)
world_merge['user_state'].fillna('Unknown', inplace=True)

# Identify the top 10 countries
top10 = world_merge.sort_values(by='nb_reviewers', ascending=False).head(10)
top10 = top10[['name', 'nb_reviewers']]
top10.index = np.linspace(1, 10, 10).astype(int)
top10.nb_reviewers = top10.nb_reviewers.astype(int)

# Display the Top 10 countries (considering US states) with the most reviewers
print('Top 10 countries (considering US states) with the most reviewers:')
display(top10)

# Plot the choropleth map
fig, ax = plt.subplots(1, 1, figsize=(15, 7))
world_merge.boundary.plot(ax=ax, color='black', linewidth=0.5)
world_merge.plot(column='nb_reviewers', ax=ax, legend=True,
                 norm=LogNorm(vmin=1, vmax=world_merge['nb_reviewers'].max()),
                 legend_kwds={'label': "Number of Reviewers by Country", 'orientation': "vertical"},
                 cmap='Blues')

# Remove the axis
ax.set_title('Distribution of the reviewers in the World')
ax.set_axis_off()
plt.show()


We can see that **the majority of reviewers are concentrated in the USA**. This observation will be a crucial factor in our subsequent analysis.

### **Relative distance between Reviewers and Breweries**

Now that we have examined the geographical distribution of breweries and reviewers, our next focus is to **explore the relative distance between the reviewer and the brewery for each review**. This analysis aims to provide insights into **how the popularity of a brewery is distributed globally**.

To achieve this, we begin by calculating the centroid of each country (or state in the case of US). Subsequently, we add these centroid values to both the users and breweries dataframes

In [ ]:
# Add centroid to world dataset
world_df = world_with_US_states.copy(deep=True)
world_df['centroids'] = world_df['geometry'].centroid.to_crs(32633)
world_df = world_df.set_index('name')
display(world_df)

In [ ]:
# Create a dataframe of users and breweries with centroids

# Merge breweries with state centroids
brew_cen = pd.merge(breweries_loc, world_df, left_on=['brewery_state'], right_on=['name'], how='inner')
brew_cen.rename(columns={'centroids': 'brewery_centroid'}, inplace=True)

# Merge breweries with country centroids
brew_cen = pd.merge(brew_cen, world_df, left_on=['brewery_location'], right_on=['name'], how='inner')
brew_cen.rename(columns={'centroids': 'brewery_centroid_country'}, inplace=True)

# Drop unnecessary columns
brew_cen = brew_cen.drop(['geometry_x', 'geometry_y', 'nbr_beers', 'brewery_name'], axis=1)

# Merge users with state centroids
user_cen = pd.merge(users_loc, world_df, left_on=['user_state'], right_on=['name'], how='inner')
user_cen.rename(columns={'centroids': 'user_centroid'}, inplace=True)

# Merge users with country centroids
user_cen = pd.merge(user_cen, world_df, left_on=['user_location'], right_on=['name'], how='inner')
user_cen.rename(columns={'centroids': 'user_centroid_country'}, inplace=True)

# Drop unnecessary columns
user_cen = user_cen.drop(['geometry_x', 'geometry_y', 'nbr_ratings', 'nbr_reviews', 'user_name', 'joined'], axis=1)

# Display a sample of the resulting dataframes
display(user_cen.sample(3))
display(brew_cen.sample(3))


In [ ]:
# Merge with reviews
review_brew_user = reviews_filt[['brewery_id','user_id']].copy(deep=True)
review_brew_user = pd.merge(review_brew_user,brew_cen,on=['brewery_id'],how='inner')
review_brew_user = pd.merge(review_brew_user,user_cen,on=['user_id'],how='inner')
display(review_brew_user.sample(3))

Now that we have a dataframe containing every review, along with the centroid of the reviewed brewery and the centroid of the user's residence, we can proceed to **compute the distance between the reviewer and the brewery**.

In [ ]:
rev_distance = review_brew_user.copy(deep=True)
rev_distance['distance_state'] = rev_distance.apply(lambda row: row['brewery_centroid'].distance(row['user_centroid'])/1000, axis=1)

rev_distance['distance_country'] = rev_distance.apply(lambda row: row['brewery_centroid_country'].distance(row['user_centroid_country'])/1000, axis=1)

display(rev_distance.sample(10))
#rev_distance['distance'].describe()

In [ ]:
# Plot a histogram of the distances between reviewer and brewery locations
plt.hist(rev_distance['distance_country'], bins=np.linspace(0, 50000, 500), log=True)

# Set x and y labels and the title
plt.xlabel('Distance between reviewer location and brewery location [km]')
plt.ylabel('Number of reviews')
plt.title('Reviewers relative distance to breweries')

# Display the plot
plt.show()

### **Comparison of Relative Distance with Size and Popularity Metrics**

In [ ]:
# Merge distance and metrics dataframes on 'brewery_id'
metric_distance = pd.merge(rev_distance[['brewery_id', 'distance_country', 'distance_state']],
                          df_metrics[['brewery_id', 'size_metrics', 'popularity_metrics']],
                          on=['brewery_id'], how='inner')

# Group by 'brewery_id' and calculate the median
metric_distance = metric_distance.groupby('brewery_id').median()

display(metric_distance)

In [ ]:
# Create subplots
fig, axs = plt.subplots(1, 2, figsize=(12, 4))

# Plot scatter for 'size metrics' in the first subplot
axs[0].scatter(metric_distance['size_metrics'], metric_distance['distance_country'], color='blue')
axs[0].set_title('Distribution of the breweries based on size metrics')
axs[0].set_ylabel('Mean distance of reviewers')
axs[0].set_xlabel('Size metric')

# Plot histogram for 'popularity metrics' in the second subplot
axs[1].scatter(metric_distance['popularity_metrics'], metric_distance['distance_country'], )
axs[1].set_title('Distribution of the breweries based on popularity metrics')
axs[1].set_ylabel('Mean distance of reviewers')
axs[1].set_xlabel('Popularity metric')

axs[0].set_xscale('log')
# Adjust layout
plt.tight_layout()

# Display the subplots
plt.show()

### **Merge reviews with Users dataframe**

In [ ]:
reviews_aug = pd.merge(reviews_filt, users_loc, on=['user_id','user_name'], how='inner')
display(reviews_aug)

### **Merge Reviews and Users with breweries dataframe**

In [ ]:
df_RBU = pd.merge(reviews_aug, breweries_loc, on=['brewery_id','brewery_name'], how='inner')
display(df_RBU)
print('Number of NaN by category:')
print(df_RBU.isna().sum())

### **Temoral Analysis on ratings and the number of reviews**

With existing overview of the dataset, the next step is to investigate the temporal evolution of the beer market from the perspective of user ratings and reviews. Aligned with our interest in how brewery sizes matter, we pay specific attention to **how the ratings and reviews are distributed among breweries**.

In [ ]:
# Take a subset columns of the dataframe and rename columns for clarification
merged = df_RBU[['brewery_name', 'brewery_id', 'date','rating', 'text', 'nbr_ratings', 'nbr_reviews', 'joined']].copy()

merged.rename(columns={'nbr_ratings': 'nbr_ratings_user',
                       'nbr_reviews': 'nbr_reviews_user',
                       'date': 'date_review',
                       'joined': 'joined_user'
                      }, inplace=True)

# Extract year from the datetime
# date_format = '%Y-%m-%d %H:%M:%S'
merged['joined_user_year'] = merged.joined_user.apply(lambda x: x.year)
merged['date_review_year'] = merged.date_review.apply(lambda x: x.year)

# merge by the [year, brewery]
merged_brewery_year = merged.groupby(['date_review_year','brewery_id']).apply(lambda x: pd.Series({
    'brewery_nbr_reviews': x.size,
    'brewery_avg_rating': x.rating.mean(),
    'brewery_rating_list': x.rating.tolist()
})).reset_index()

print('Data for each brewery per year')
merged_brewery_year.head()

In [ ]:
# merge by the year of ratings for all breweries, preserving both macro (for breweries) and micro (equally treating reviews) info
'''
For each year::
nbr_reviews: #reviews received for all breweries
brewery_nbr_reviews: a list of #reviews for each brewery 
macro_avg_brewery_rating: the macro average rating of each brewery
brewery_avg_rating_list: the list of each brewery's average rating
ratings: all received rating scores
'''
merged_year = merged_brewery_year.groupby(['date_review_year']).apply(lambda x: pd.Series({
    'nbr_reviews': x.brewery_nbr_reviews.sum(),
    'brewery_nbr_reviews': x.brewery_nbr_reviews.tolist(),
    'macro_avg_brewery_rating': x.brewery_avg_rating.mean(),
    'brewery_avg_rating_list': x.brewery_avg_rating.tolist(),
    'ratings': x.brewery_rating_list.sum()
})).reset_index()

merged_year['micro_avg_rating'] = merged_year.ratings.agg(lambda x: np.mean(x))
print('Data for all breweries per year')
merged_year.head()

In [ ]:
# statistics for visualization
stats_by_year = pd.DataFrame({
    'year': merged_year.date_review_year,
        'macro_average': merged_year.macro_avg_brewery_rating,
        'macro_std': merged_year.brewery_avg_rating_list.apply(lambda x: np.std(x)),
        'micro_average': merged_year.micro_avg_rating,
        'micro_std': merged_year.ratings.apply(lambda x: np.std(x)),
        'nbr_reviews': merged_year.nbr_reviews,
        'average_nbr_reviews': merged_year.brewery_nbr_reviews.apply(lambda x: np.mean(x)),
        'nbr_reviews_std': merged_year.brewery_nbr_reviews.apply(lambda x: np.std(x)),
        'nbr_reviews_percentage': merged_year[['nbr_reviews','brewery_nbr_reviews']].apply(lambda x: [y / x['nbr_reviews'] for y in x['brewery_nbr_reviews']], axis=1)
})
stats_by_year['percentage_mean'] = stats_by_year.nbr_reviews_percentage.apply(lambda x: np.mean(x))
stats_by_year['percentage_median'] = stats_by_year.nbr_reviews_percentage.apply(lambda x: np.median(x))
stats_by_year['percentage_std'] = stats_by_year.nbr_reviews_percentage.apply(lambda x: np.std(x))
stats_by_year['nbr_brewery'] = stats_by_year.nbr_reviews_percentage.apply(lambda x: len(x))

In [ ]:
plt.figure(figsize=(8, 5), dpi=100)
plt.fill_between(stats_by_year.year, stats_by_year.macro_average - stats_by_year.macro_std,
                 stats_by_year.macro_average + stats_by_year.macro_std, alpha = 0.2, color = 'C1')
plt.plot(stats_by_year.year, stats_by_year.macro_average, color = 'C1', label='_nolegend_')
plt.fill_between(stats_by_year.year, stats_by_year.micro_average - stats_by_year.micro_std,
                 stats_by_year.micro_average + stats_by_year.micro_std, alpha = 0.2, color = 'C0')
line_micro = plt.plot(stats_by_year.year, stats_by_year.micro_average, color = 'C0', label='_nolegend_')

plt.ylim([2, 5])
plt.xlim([1996, 2017])
plt.xticks([1996,1998,2000,2002,2004,2006,2008,2010,2012,2014,2016,2017])
plt.legend(['Macro Average','Micro Average'])
plt.title('The average of all ratings vs the average across breweries')
plt.xlabel('Year')
plt.ylabel('Ratings')

From the trend average of ratings, we observe that the micro average is generally higher than the macro average across the breweries. It indicates scenario such as: a few breweries receiving relatively higher amount of high ratigns. This motivates us to explore further the data with respect to different sizes of popularities of breweries.

In [ ]:
t = stats_by_year.year

fig, ax1 = plt.subplots(figsize=(9, 5), dpi=100)

# color = 'C1'
ax1.set_xlabel('Year')
ax1.set_xticks([1996,1998,2000,2002,2004,2006,2008,2010,2012,2014,2016,2017])
ax1.set_yscale('log')
ax1.set_xlim([1996,2017])
ax1.set_ylabel('Percentage #ratings for breweries', color='chocolate')
ax1.plot(stats_by_year.year, stats_by_year.percentage_mean, color = 'orange')
ax1.plot(stats_by_year.year, stats_by_year.percentage_median, color = 'olive')
plt.legend(['Mean of review percentages','Median of review percentages'], loc = 'upper center')
# ax1.errorbar(stats_by_year.year, stats_by_year.percentage_mean,
#              yerr = stats_by_year.percentage_std,
#              capsize= 3, color = color)
ax1.tick_params(axis='y', labelcolor='chocolate')

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color2 = 'tab:blue'
ax2.set_ylabel('#reviews', color=color2) 
ax2.set_xlim([1996,2017])
ax2.plot(stats_by_year.year, stats_by_year.nbr_brewery, color = color2)
ax2.tick_params(axis='y', labelcolor=color2)
plt.legend(['#reviews'], loc='center')


fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.title('The trend of percentage of reviews that breweries received & the total number of reviews')
plt.show()

The number of reviews is increasing along the years, possibly due to the development of internet and the brewing process. The divergence of mean and median of percentage of reviews received by breweries again indicate an unenvenly distribution across breweries。

From the decreasing curves of means and medians, we have two assumptions that should be validated later during mileston 3:
* The number of breweries competing in the beer market is growing
* In its early years, the BeerAdvocate website collected ratings for only a few breweries.


In [ ]:
stats_by_year_long = stats_by_year[stats_by_year.year >= 1997][['year','nbr_reviews_percentage']].copy()
stats_by_year_long = stats_by_year_long.explode('nbr_reviews_percentage')
# df_long['grades'] = pd.to_numeric(df_long['grades'])
plt.figure(figsize=(10, 4), dpi=100)
ax = sns.boxplot(data=stats_by_year_long, x='year', y='nbr_reviews_percentage')
ax.set_xlabel('Year')
ax.set_ylabel('Percentage of #reviews')
ax.set_title('The distribution of percentage of reviews received by breweries')

From the boxplot, it is interesting to observe massive number outliers with percentage of reviews higher than average, especially after the year of 2000. The boxplot indicates that while the mojority of breweries receive nearly zero reviews on BeerAdvocate website, there are a few breweries receiving much higher number of reviews. Therefore, we research question regarding this phenomenon is: 

**Does monopoly power dominates the beer industry? If yes, do those monopoly share common sizes and popularities?** 